## Project: Bank Analysis - Tweets 
Using Twint to scrape tweets 

Use Optimus to clean tweets 


# Quick Twint code
Github - https://github.com/twintproject/twint

All functions - https://github.com/twintproject/twint/wiki/Configuration
### setup 
c.twint.Config() 

#### set username
c.Username = "realDonaldTrump"

#### set phrase search
c.Search = "great"

#### customise output
c.Custom["tweet"] = ["id"]           --- assign column names

c.Custom["user"] = ["bio"]

c.Limit = 1                          ---- limit to batches (unknown size)

c.Since = "2019–04–29"

c.Until = "2020–04–29"

#### Pandas 
c.Pandas = True

    --------once run save to dataframe
    
    ---- Tweets 
    df = twint.storage.panda.Tweets_df
    
    ---- followers 
    df = twint.storage.panda.Follow_df
    
    df = Followers_df['followers'][username]

#### Write output 
c.Store_csv = True

c.Output = "test.csv"

### Execute determines how its run- using setup above
twint.run.Search(c)    --- will run search for all tweets with the above

twint.run.Profile(c)   ---- will run against profile - return only this profiles tweets

twint.run.Followers(c) ---- get follower info 


config.Since = "2019–04–29"
config.Until = "2020–04–29"
config.Store_json = True


In [50]:
import twint
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

#from optimus import Optimus                   


In [51]:
# Configure for CSV 
'''
c = twint.Config()
c.Search = "Standard Bank"

## change specifics
c.Limit = 1
c.Store_csv = True
c.Output = "StandardBank.csv"
'''

'\nc = twint.Config()\nc.Search = "Standard Bank"\n\n## change specifics\nc.Limit = 1\nc.Store_csv = True\nc.Output = "StandardBank.csv"\n'

In [52]:
# Pandas 
c = twint.Config()
c.Search = "Standard Bank"

## change specifics
c.Limit = 1
c.Pandas = True
c.Pandas_clean= True
c.lang = "en"
#c.Output = "StandardBank.csv"

In [53]:
#run Search 
twint.run.Search(c)

1420693350404005888 2021-07-29 12:31:04 +0200 <Yfm> WIN R1500 with “Who killed the beat” on The Lady on Drive with the @StandardBankArt Jazz Festival!🎷  1. Choose a jazz note here🎵:  https://t.co/Bgx3CGyCOW 2. Put your creative spin on it!🎤 3. Post a video or voice recording with the #NAF2021 &amp; tag @yfm  Standard Bank #ItCanBe  https://t.co/MAj17He54A
1420692310841561091 2021-07-29 12:26:56 +0200 <StandardBankZA> @GuruAfrica1 Apologies on that end, please note that a payments  from Standard Bank to Standard Bank will reflect with in 24hrs, please advise at what time was the payment made?
1420690926398214146 2021-07-29 12:21:26 +0200 <GuruAfrica1> @StandardBankZA I made a payment, standard bank to standard bank, but you won't clear it, have to wait up to 24hrs before it reflects. You have immediate payment options for other banks, but you don't have one in place for your own. This is rubbish
1420687624528961539 2021-07-29 12:08:19 +0200 <artsfestival> Only 3 more days to catch the s

## Pandas and cleaning 

In [54]:
#set pandas
df_pd = twint.storage.panda.Tweets_df

In [55]:
df_pd.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [72]:
#Export to csv
#df_pd.to_csv("test_post.csv")

In [57]:
new_df = df_pd[["date", "username", "tweet", "hashtags", "nlikes","search"]]

In [58]:
new_df.head(5)

,date,username,tweet,hashtags,nlikes,search
0,2021-07-29 12:31:04,Yfm,WIN R1500 with “Who killed the beat” on The La...,"[naf2021, itcanbe]",1,Standard Bank
1,2021-07-29 12:26:56,StandardBankZA,"@GuruAfrica1 Apologies on that end, please not...",[],0,Standard Bank
2,2021-07-29 12:21:26,GuruAfrica1,"@StandardBankZA I made a payment, standard ban...",[],0,Standard Bank
3,2021-07-29 12:08:19,artsfestival,Only 3 more days to catch the stellar line-up ...,[],3,Standard Bank
4,2021-07-29 12:05:30,NAROMARTIN,@StandardBankNa @paypulse standard bank Ongwed...,[],0,Standard Bank


### Cleaning tweet data 


In [74]:
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()


Data_to_clean = new_df.tweet[:]
cleaned_list = []

for t in Data_to_clean:
    cleaned_list.append(tweet_cleaner(t))
cleaned_list

['win r with who killed the beat on the lady on drive with the jazz festival choose a jazz note here put your creative spin on it post a video or voice recording with the naf tag standard bank itcanbe',
 'apologies on that end please note that a payments from standard bank to standard bank will reflect with in hrs please advise at what time was the payment made',
 'i made a payment standard bank to standard bank but you won t clear it have to wait up to hrs before it reflects you have immediate payment options for other banks but you don t have one in place for your own this is rubbish',
 'only more days to catch the stellar line up at the standard bank arts jazz festival check it all out here photos by mark wessels mqiki',
 'standard bank ongwediva homeloan service is not good the guy is always working at home but never answer our call client apply since june but till now is not finalize but she is finacial approved what can we do to follow up this matter',
 'standard bank will mail m

In [77]:
clean_df = pd.DataFrame(cleaned_list,columns=['tweet'])
clean_df['target'] = 0
clean_df.head()

,tweet,target
0,win r with who killed the beat on the lady on ...,0
1,apologies on that end please note that a payme...,0
2,i made a payment standard bank to standard ban...,0
3,only more days to catch the stellar line up at...,0
4,standard bank ongwediva homeloan service is no...,0
